In [ ]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from qdrant_client.http import models
from myAgent.qdrant import qdrant
from myAgent.ollama import embed
import json

tab = pd.read_csv("pkm.csv")
tab

In [ ]:
qdrant.delete_collection(collection_name="pokemons")
qdrant.create_collection(
    collection_name="pokemons",
    vectors_config=models.VectorParams(
        size= 1024,
        distance=models.Distance.COSINE
    )
)
qdrant.get_collections()

In [ ]:
verbose = [1,4,7,25,151]
for id in tab["id"].unique():
    
    tmp = tab[tab["id"] == id]
    nom = tmp["nom"].iloc[0]
    type = tmp["type"].iloc[0]
    description = ""
    for j in range(len(tmp)):
        description = description + tmp["description"].iloc[j] + " "
    description = description.replace("\r", "").replace("\n", " ")
    prompt = nom + " est un " + type + ". " + description

    if id in verbose:
        print(prompt + "\n\n")

    # prompt_en = translate(prompt_fr)
    #
    # if id in verbose:
    #    print(prompt_en + "\n\n")

    vector = embed(prompt)
    
    qdrant.upsert(
        collection_name="pokemons",
        points=[
            models.PointStruct(
                id = int(id),
                vector = vector,
                payload={
                    "numero": int(id),
                    "nom": nom,
                    "type": type,
                    "description": description,
                }
            )
        ]
    )

In [ ]:
all_points = qdrant.scroll(
    collection_name="pokemons",
    limit=10000,
    with_payload=True,
    with_vectors=True
)[0]
embeddings = []
ids = []
for point in all_points:
    embeddings.append(point.vector)
    ids.append(point.payload.get("nom", point.id))
embeddings = np.array(embeddings)
pca = PCA(n_components=2)
pca_result = pca.fit_transform(embeddings)
explained_variance = pca.explained_variance_ratio_ * 100  # Convert to %
plt.figure(figsize=(10, 8))
plt.scatter(pca_result[:, 0], pca_result[:, 1], alpha=0.7)
for i, name in enumerate(ids):
    plt.annotate(name, (pca_result[i, 0], pca_result[i, 1]), fontsize=8, alpha=0.8)
plt.xlabel(f"PC1 ({explained_variance[0]:.2f}%)")
plt.ylabel(f"PC2 ({explained_variance[1]:.2f}%)")
plt.title("PCA Projection of Pokémon Embeddings")
plt.grid(True)
plt.show()